<a href="https://colab.research.google.com/github/uh2000/ML/blob/main/Scaling_Unbalanced_Data_Introduction_Intro_and_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Theory

In [ ]:
# SETUP
# install python at version 3.10
!apt-get install python3.10

# update symbolic links to the newly installed python version
!ln -sf /usr/bin/python3.10 /usr/bin/python
!ln -sf /usr/bin/python3.10 /usr/bin/python3

# install numpy 1.23.5
%pip install numpy==1.23.5
# install scikit 1.2.2
%pip install scikit-learn==1.2.2

### Preprocessing

One of the tasks of a data scientist is to guide the training process of  Machine Learning algorithms. <br>
This operation does require the definition of a training strategy, which might involve some preprocessing operations. <br>
Preprocessing operations are meant to ease the training phase of our models, and sometimes they are essentials! <br>
In today's exercises, we will see a practical example. <br>

Now we show you some Sklearn preprocessing functions <a href = "https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing"> [link] </a>.

Usually, scaling operations are performed column-wise, i.e., each column is standardized independently.

In [ ]:
import numpy as np

#define our array
data = np.array([[0, 0], [0, 0], [1, 2], [1, 1]])
print(data)

A first example of preprocessing is the MinMaxScaler <a href = https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler> [link] </a>.

The MinMaxScaler transforms the features in a fixed range scale, for example $[0, 1]$ (default) <br>

The MinMaxScaler is defined as follows:

    min = 0
    max = 1
    X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    X_scaled = X_std * (max - min) + min

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#define the scaler
scaler1 = MinMaxScaler()

#fit the scaler
scaler1.fit(data)

print(scaler1.transform(data))

Similarly, we can use the the StandardScaler <a href = "https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler"> [link] </a>. <br>

This scaler standardizes features by removing the mean and scaling to unit variance. For example, given an array $X$, the function outputs the following
$X_{std} = \frac{(X - \mu)}{\sigma}$

In [ ]:
from sklearn.preprocessing import StandardScaler

#define the scaler
scaler2 = StandardScaler()

#fit the scaler
scaler2.fit(data)

print(scaler2.transform(data))

#we can plot also the mean of each axis
print(scaler2.mean_)

If you apply preprocessing techniques, you need to use them on every sets (i.e., training, validation, and testing) **consistently**. <br>
When using preprocessing techniques, you need to apply the *fit* operation to the **training set only**. <br>
The validation set can be used to find the best hyperparameter setting of the preprocessing function (if any). <br>
The test set, instead, cannot be used for any reason. <br>
We can now see a simple example of what might go wrong if do not respect such a requirement.

In [ ]:
#we first define a training array
X_tr = np.array([[0, 1, 2, 3, 4, 5, 6]]).T

#the ground truth is defined as a binary classification task, where the sample
# is 0 if <=3, 1 otherwise
y_tr = np.array([0, 0, 0, 0, 1, 1, 1])

#we define the test set with the same rules
X_te = np.array([[0, 0, 3, 20]]).T
y_te = np.array([0, 0, 0, 1])

In [ ]:
X_tr.shape

In [ ]:
#we might want to scale all of the training data in the range [0, 1]
scaler_tr = MinMaxScaler()
scaler_tr.fit(X_tr)
X_tr_scaled = scaler_tr.transform(X_tr)
print(X_tr_scaled)

A classifier might define a rule that, given a scaled point $x_i$, returns 0 if it is lower or equale than 0.5, 1 otherwise.  

In [ ]:
#we now wrongly fit a new scaler over the test set.
#let's see what happens
print(X_te)
scaler_te = MinMaxScaler()
scaler_te.fit(X_te)
X_te_scaled = scaler_te.transform(X_te)
print(X_te_scaled)

The classifier we previously defined might work well in most cases, but it misclassify $x_i = 3$: now its scaled value is 0.15 and not 0.5. <br>
If we use the correct process, i.e., transforming the testing data with the scaler fitted on the training data, the output is correct.

In [ ]:
#we now wrongly fit a new scaler over the test set.
#let's see what happens
print(X_te)
X_te_scaled = scaler_tr.transform(X_te)
print(X_te_scaled)

### Undersampling / Oversampling

Sometimes we might face unbalanced datasets. <br>
In classification tasks, this means that the number of samples per class are not equal among the classes. <br>
These dataset can produce *deceiving* results, and today we are going to experiment with an ad hoc example. <br>

Let $X, y$ be a dataset defined over two classes $(c_0, c_1)$.
Let $c_0$ be the minority class, and $c_1$ the majority one. <br>
We might face different unbalance levels, for example weak unbalance (e.g., $c_0 = 100$ and $c_1 = 110$), or heavily unbalance (e.g., $c_0 = 100$ and $c_1=1000$). <br>
In general, when we have unbalanced data, we can apply two mitigation strategies:


1.   *Oversampling*: we use a generator $G$ to produce new samples belonging to $c_0$. In this way, we make $c_0$ bigger and closer to $c_1$.
2.   *Undersampling*: we use a generator $G$ to select a subset of samples belonging to $c_1$. In this way, we make $c_1$ smaller and closer to $c_0$.

One simple way to implement such strategies in sci-kit learn is to use the **class_weight** parameter of many learning algorithms ([Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression) and [Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html?highlight=perceptron#) are among these). **class_weight** multiplies the error, during learning, on every example of that class. We can use that to weight the errors on the minority class such that overall they count as much as the ones on the majority class (see the explanation of the 'balanced' value of the parameter **class_weight**). <br>
We now turn our attention to techniques that directly change the data at preprocessing time.
We can find several techniques to define $G$. Today we show an example of oversampling and undersampling $G$. Remember: when defining $G$ we can explicitly define the magnitude of the sampling strategy. <br>
For example, in the case of oversampling:


*   100% balancy: from $c_0 = 100$ and $c_1=1000$ to $c_0 = 1000$ and $c_1=1000$.
*   50% balancy: from $c_0 = 100$ and $c_1=1000$ to $c_0 = 500$ and $c_1=1000$.


In our exercises we use generators $G$ developed by *imbalance learn* library, a library written on top of *sklearn* <a href = "https://imbalanced-learn.org/stable/index.html"> [link] </a>. <br>
We are going to use [SMOTE](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html?highlight=smote#imblearn.over_sampling.SMOTE) (an oversampler), and [RandomUnderSampler](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html?highlight=randomundersampler) (an undersampler).




In [ ]:
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

#we generate a two-classes dataset. 1000 samples.
# c0 = 100 samples, c1 = 900 samples
X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative= 1, n_redundant= 0,
                           flip_y=0, n_features=1, n_clusters_per_class=1, n_samples=1000, random_state=123)

#print the first 10 samples
print(X[:10], '\n' ,y[:10])

#we can see the distribution of the two classes
print(len(X))
print(f"Class 0:{np.sum(y == 0)}")
print(f"Class 1:{np.sum(y == 1)}")

In [ ]:
#we can try to use the undersampling strategy
#by using 0.5 as ratio, we ask the classifier to make the minority class 50% of the size of the majority one
#here the minority class size is fixed. We change the majority
undersampler = RandomUnderSampler(random_state = 123,
                                  sampling_strategy= 0.5)

#resample
X_under, y_under = undersampler.fit_resample(X, y)

#we can see the distribution of the two classes
print(len(X_under))
print(f"Class 0:{np.sum(y_under == 0)}")
print(f"Class 1:{np.sum(y_under == 1)}")

In [ ]:
#we can try with the oversampling strategy
#by using 0.5 as ratio, we ask the classifier to make the minority class 50% of the size of the majority one
# here the majority class size is fixed. We vary the minority
oversampler = SMOTE(random_state = 123, sampling_strategy= 0.5)

#resample
X_over, y_over = oversampler.fit_resample(X, y)

#we can see the distribution of the two classes
print(len(X_over))
print(f"Class 0:{np.sum(y_over == 0)}")
print(f"Class 1:{np.sum(y_over == 1)}")

--------------
# Exercises

### Exercise 1: Scaling

We start by loading the dataset that we use in our exercise.


In [ ]:
from sklearn.datasets import load_wine #load our target dataset
import numpy as np

#load the dataset
dataset = load_wine()

#extract X and y
X = dataset.data
y = dataset.target

The dataset contains three classes:

In [ ]:
print(y)

We now have a look on how the data looks like. <br>
Print the X and y shapes.


In [ ]:
#
#   complete EX 2.1.1 here
#

The dataset contains $178$ samples and $13$ features. <br>
The first thing to do, is to create the train, val, and test partitions. <br>

To do so, start by defining a split *X_train_val* and *X_test*, with $80\%$ of the samples in the *X_train_val* set. <br>
Then, get *X_train* and *X_val*, where *X_train* contains the $90\%$ of the samples. <br>

Set the **random_state** to $123$ in this phase.


In [ ]:
#
#   complete EX 2.1.2 here
#

Now define a LogisticRegression with default parameters (and **random state** = $123$), fit it on the training data, and evaluate both on the training and validation sets. <br>
Use the *accuracy score* as the evaluation metric. <br>

In [ ]:
#
#   complete EX 2.1.3 here
#
# expected output: 0.968503937007874 0.8666666666666667
#

The previous cell returns the following message:

    ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

    Increase the number of iterations (max_iter) or scale the data as shown in: https://scikit-learn.org/stable/modules/preprocessing.html

    Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
    extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
    (0.9824561403508771, 0.8666666666666667)

Sklearn suggest you to increase the max number of iterations, or to scale the data. <br>
We go for the second way. <br>
Using $X_{tr}$, fit a *standard scaler*, and transform $X_{tr}$, $X_{val}$, and $X_{te}$ using the fitted scaler.

In [ ]:
#
#   complete EX 2.1.4 here
#
# The new variables are called X_train_scaled, X_val_scaled, and X_test_scaled

Print $\mu$ of the scaler.

In [ ]:
#
#   complete EX 2.1.5 here
#

We now check if the preprocessing solved our issue encountered in Ex 2.1.3. <br>
Train the logistic regression with default parameters (and random state =  123) on the scaled data and print the accuracy on the training and validation sets.

In [ ]:
#
#   complete EX 2.1.6 here
#

# Exercise 2.2: Dealing with unbalanced data

We first define a new ad hoc dataset. This time we fix the random_state to 1.

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt

X_toy, y_toy = datasets.make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)

Print the number of samples, and the number of features. <br>

Split the data: first create a test set ($20\%$), and then split the remaining data into training and validation sets ($90\%$ training size, $10\%$ validation set size). Again, set the **random_state** to $123$.

In [ ]:
#
#   complete EX 2.2.1 here
#

Train a Logistic Regression with default parameters (**random state** = $123$) on the training set, and evaluate it on the validation set. <br>
Use the *accuracy score* as evaluation metric.  

In [ ]:
#
#   complete EX 2.2.2 here
#

We just obtained a wonderful score! Almost perfect prediction. <br>
However, in datascience, we always need to verify our scores. <br>
We can start by counting the number of samples of each class contained in the training and validation sets (separately).

In [ ]:
#
#   complete EX 2.2.3 here
#

There is a high unbalance! For example, in the validation we have 791 samples belonging to class 0, and only 9 to class 1! Can this affect our scores? <br>
To understand how good is an accuracy of $0.99$, do the following:
create an array $y_{dummy} = 0$, where $|y_{dummy}| = |y_{val}|$. <br>
$|x|=n$ is the cardinality of x, in this example the vector x has n elements.
<br>
Once you defined $y_{dummy}$, compute the validation accuracy between the true validation labels (i.e., $y_{val}$) and $y_{dummy}$.

In [ ]:
#
#   complete EX 2.2.4 here
#

Ok, that is not exactly amazing. <br>
The accuracy score is maybe not the best metric for unbalanced data. <br>
Let's use the [$F_1$ score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score) instead. <br>
Print both validation F$_1$ of the $lr$ predictions and the $y_{dummy}$ with respect to the true labels.

In [ ]:
#
#   complete EX 2.2.5 here
#
# Expected output: 0.61 for the logistic regression, and 0.0 for the dummy array.

F$_1$ gives a more useful picture of our classifiers. <br>
Now we just need to improve the classification performance. <br>
We start by plotting the data. How does training data look like?


In [ ]:
# scatter plot of examples by class label
from collections import Counter
from numpy import where

counter = Counter(y_train)

for label, _ in counter.items():
	row_ix = where(y_train == label)[0]
	plt.scatter(X_train[row_ix, 0], X_train[row_ix, 1], label=str(label))
plt.legend()
plt.show()

There is a big blue cluster, and a few orange points. <br>
Let's use SMOTE to oversample the minority class. <br>
First, let's apply SMOTE with **sampling_strategy**=$0.05$ and **random state** = $123$ over the training set, and obtain new variables *X_train_over*, *y_train_over*.

In [ ]:
#
#   complete EX 2.2.6 here
#

After the oversampling, print the number of samples contained in every class of the training. Is the balancy situation improved compared to the raw training set?

In [ ]:
#
#   complete EX 2.2.7 here
#

We can see how the training data changed.

In [ ]:
counter = Counter(y_train_over)

for label, _ in counter.items():
	row_ix = where(y_train_over == label)[0]
	plt.scatter(X_train_over[row_ix, 0], X_train_over[row_ix, 1], label=str(label))
plt.legend()
plt.show()

SMOTE creates novel artificial examples. Here is how it works (from the [paper](https://arxiv.org/pdf/1106.1813.pdf) where it was first described) in a nutshell:  
```
The minority class is over-sampled by taking each minority class sample and introducing synthetic examples along the
line segments joining any/all of the k minority class nearest neighbors.
Define a new Logistic Regression (random state = 123), train it over the oversampled
```

Define a new Logistic Regression with **random_state** = $123$, train it over the oversampled training data, and calculate the new validation F$_1$.


In [ ]:
#
#   complete EX 2.2.8 here
#


Definitely better! Good job. <br>
We can now try another way to solve the unbalance problem: instead of increasing the number of samples belonging to the minority class (i.e., class 0), we reduce the number of sample of the majority one (i.e., class 1). <br>
Remember: the fit operations can be done only in the training data. <br>

Use the *RandomUnderSampler* with default parameters to undersample the training data. <br>
Call the new variables and obtain new variables X_train_under, y_train_under.
Print the distribution of the two undersampled classes.

In [ ]:
#
#   complete EX 2.2.9 here
#

Train a LogisticRegression using the undersampled training data, and evaluate with the F$_1$ on the validation set.

In [ ]:
#
#   complete EX 2.2.10 here
#

We do not obtain a great performance, with only 73 samples per class. <br>
Let's try another sampling variation. <br>
In our first sampling solution we add samples to the minority class until the size is the same of the majority one, and in the second case we did the opposite, by reducing the majority class. <br>
Sampling solutions allow us to control the amount of *unbalancement* we want. <br>

For example, we can create a new undersampling strategy with **sampling_strategy = 0.5**. This ratio indicates that the minority class size will be $1/2$ of the majority class size. <br>
Print the distribution of the new training data, then train a new LogisticRgression and evaluate it (with F$_1$) on the validation data. <br>
Play with the value of **sampling_strategy**, what is the best F$_1$ that you can get on the validation set?

In [ ]:
#
#   complete EX 2.2.11 here
#